In [ ]:
!pip install folium geopy rtree matplotlib mapclassify 
!pip install geopandas


In [ ]:
import folium
import geopandas as gpd
import os
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, Fullscreen, MousePosition

In [ ]:
def load_shapefile(name, cols, index=None, rename=None):
    path = os.path.abspath(f"./data/{name}.zip")
    df = gpd.read_file(f'zip://{path}')

    df = df[cols]
    if rename:
        df = df.rename(columns=rename)
    if index:
        df = df.set_index(index)
    return df.to_crs(epsg=4326)
    

def add_barrios(m):
    for name, r in barrios.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'fillColor': 'orange'})
        folium.Popup(f"<b>Barrio:</b> <span style='white-space: nowrap;'>{name}</span>").add_to(geo_j)
        geo_j.add_to(m)
    m



def get_map(withbarrios=True):
    #tiles maps: https://github.com/python-visualization/folium/tree/main/folium/templates/tiles
    # Create a base map
    m = folium.Map(location=[40.422, -3.696], tiles='stamentoner', zoom_start=12)
    m.add_child(Fullscreen())
    m.add_child(MousePosition())
    
    if withbarrios:
        add_barrios(m)
    
    return m

In [ ]:
# Barrios Dataset
barrios = load_shapefile("barrios", ["NOMBRE", "geometry"], index="NOMBRE")

barrios

In [ ]:
distritos = load_shapefile("distritos", ["DISTRI_MAY", "geometry"], rename={"DISTRI_MAY": "NOMBRE"}, index="NOMBRE")

distritos

In [ ]:
# Bicimad dataset
import json

with open('./data/bicimad-estaciones-202106.json') as f:
    raw = json.load(f)
    
stations = [{
          "type": "Feature",
          "geometry": { "type": "Point", "coordinates": [float(station["longitude"]), float(station["latitude"])] },
          "properties": station
       } for station in raw["stations"]]

bicimad = gpd.GeoDataFrame.from_features(stations)
bicimad = bicimad.set_crs("EPSG:4326")

bicimad

In [ ]:
# Show bicimad stations as makers
m = get_map(True)

for name, row in bicimad.iterrows():
    popup = folium.Popup(f"<b>Estación:</b> <span style='white-space: nowrap;'>{name}</span><br/><b>Docks:</b> {row['total_bases']}")
    Marker([row['latitude'], row['longitude']], popup=popup).add_to(m)

m

In [ ]:
# Show bicimad stations as clusters
m = get_map()

mc = MarkerCluster()
for idx, row in bicimad.iterrows():
    Marker([row['latitude'], row['longitude']]).add_to(mc)
m.add_child(mc)

m

In [ ]:
# Show bicimad stations as heatmap
m = get_map()
HeatMap(data=bicimad[['latitude', 'longitude']], radius=20).add_to(m)
m

In [ ]:
# calculate sum of docks per district
bicimad_distritos = gpd.sjoin(distritos, bicimad).reset_index()
bicimad_distritos = bicimad_distritos[["NOMBRE", "total_bases"]].groupby("NOMBRE").sum().squeeze()

bicimad_distritos

In [ ]:
# Show bicimad stations as Cloropeth
m = get_map(withbarrios=False)


Choropleth(geo_data=distritos.__geo_interface__,
           data=bicimad_distritos, 
           key_on="feature.id", 
           fill_color="OrRd", 
           legend_name='Bicimad Station docks per district',
           nan_fill_color = "White"
          ).add_to(m)

m

In [ ]:
# calculate sum of docks per neighbourhood
bicimad_barrios = gpd.sjoin(barrios, bicimad).reset_index()
bicimad_barrios = bicimad_barrios[["NOMBRE", "total_bases"]].groupby("NOMBRE").sum().squeeze()

bicimad_barrios

In [ ]:
# Create a base map
m = get_map(withbarrios=False)


# Add a choropleth map to the base map
Choropleth(geo_data=barrios.__geo_interface__,
           data=bicimad_barrios, 
           key_on="feature.id", 
           fill_color="OrRd", 
           legend_name='Bicimad Station docks per district',
           nan_fill_color = "LightGrey"
          ).add_to(m)

# Display the map
m